# Predicción de Ingreso Bruto de Películas
Este notebook utiliza un modelo de Random Forest Regressor para predecir el ingreso bruto (`Gross`) de películas basado en características como duración, año de lanzamiento, calificación en IMDB, puntuación en Metascore y número de votos.

In [ ]:
# Importar librerías necesariasimport pandas as pdimport matplotlib.pyplot as pltimport seaborn as snsfrom sklearn.ensemble import RandomForestRegressorfrom sklearn.model_selection import train_test_split, GridSearchCVfrom sklearn.metrics import mean_squared_error, r2_score

## 1. Carga y limpieza de datos

In [ ]:
# Cargar el dataset procesadodf = pd.read_csv('../ml-project/data/processed/processed_dataset.csv')# Dividir en características (X) y variable objetivo (y)X = df.drop(columns=['Gross'])y = df['Gross']# Dividir en conjuntos de entrenamiento, validación y pruebaX_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=42)X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

## 2. Selección del algoritmo

In [ ]:
# Seleccionamos Random Forest Regressor porque:# - Es robusto frente a datos ruidosos y no lineales.# - Puede manejar datos con diferentes escalas y no requiere normalización.# - Proporciona información sobre la importancia de las características.

## 3. Búsqueda de hiperparámetros

In [ ]:
# Definir el modelo basemodel = RandomForestRegressor(random_state=42)# Definir el espacio de búsqueda de hiperparámetrosparam_grid = {    'n_estimators': [50, 100, 200],    'max_depth': [5, 10, 20, None],    'min_samples_split': [2, 5, 10]}# Realizar búsqueda de hiperparámetros con validación cruzadagrid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=3, scoring='r2', verbose=1)grid_search.fit(X_train, y_train)# Obtener los mejores hiperparámetrosbest_params = grid_search.best_params_print("Mejores hiperparámetros:", best_params)

## 4. Entrenamiento final

In [ ]:
# Entrenar el modelo con los mejores hiperparámetrosbest_model = RandomForestRegressor(    n_estimators=best_params['n_estimators'],    max_depth=best_params['max_depth'],    min_samples_split=best_params['min_samples_split'],    random_state=42)best_model.fit(X_train, y_train)

## 5. Evaluación del modelo

In [ ]:
# Evaluar en el conjunto de validaciónval_pred = best_model.predict(X_val)val_mse = mean_squared_error(y_val, val_pred)val_r2 = r2_score(y_val, val_pred)print(f"Validación - MSE: {val_mse}, R2: {val_r2}")# Evaluar en el conjunto de pruebatest_pred = best_model.predict(X_test)test_mse = mean_squared_error(y_test, test_pred)test_r2 = r2_score(y_test, test_pred)print(f"Prueba - MSE: {test_mse}, R2: {test_r2}")

## 6. Resultados y gráficos

In [ ]:
# Gráfico de valores reales vs predichos (conjunto de prueba)plt.figure(figsize=(8, 6))sns.scatterplot(x=y_test, y=test_pred)plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], color='red', linestyle='--')plt.xlabel('Valor Real de Gross')plt.ylabel('Valor Predicho de Gross')plt.title('Valor Real vs Valor Predicho (Test Set)')plt.grid(True)plt.tight_layout()plt.show()# Importancia de característicasimportances = best_model.feature_importances_feature_names = X.columnsplt.figure(figsize=(8, 6))sns.barplot(x=importances, y=feature_names)plt.title('Importancia de Características del Modelo')plt.xlabel('Importancia')plt.ylabel('Características')plt.tight_layout()plt.show()

## 7. Predicción con nuevos datos

In [ ]:
# Crear un nuevo ejemplo para predecirnuevo_dato = pd.DataFrame([{    'Runtime': 120,          # Duración de la película en minutos    'Released_Year': 2020,   # Año de lanzamiento    'IMDB_Rating': 8.5,      # Calificación en IMDB    'Meta_score': 80,        # Puntuación en Metascore    'No_of_Votes': 500000    # Número de votos}])# Realizar la predicciónprediccion_gross = best_model.predict(nuevo_dato)print(f"Ingreso bruto predicho para el nuevo dato: ${prediccion_gross[0]:,.2f}")

## 8. Conclusiones
- El modelo Random Forest Regressor mostró un buen desempeño con un R² de validación de `{val_r2:.2f}` y de prueba de `{test_r2:.2f}`.
- Las características más importantes fueron: `{feature_names[importances.argmax()]}`.
- Este modelo puede ser utilizado para predecir el ingreso bruto de películas con datos similares.